In [27]:
import os
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf

In [81]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw_file')

df = pd.DataFrame({'gwm': [], "mfccs":[], "spectogram":[],"zcr":[],"sc":[],"ss":[],"srf":[],"sf":[],"cv":[],"cd":[], 'emotion': []})

file_num = 1
for filename in os.listdir(data_folder):
    # GWM
    signal, sample_rate = librosa.load(os.path.join(data_folder, filename), sr=None, duration=2.5, offset=0.6)
    gwm = librosa.feature.rms(y=signal).flatten() 
    
    # EMOCJA
    file_parts = filename.split('-')
    emotion = int(file_parts[2])

    #MFCCS
    mfccs = librosa.feature.mfcc(y=np.squeeze(signal), sr=sample_rate).flatten() 

    #SPECTOGRAM
    spectogram=librosa.feature.melspectrogram(y=signal, sr=sample_rate)
    spectogram_DB = librosa.power_to_db(spectogram, ref=np.max).flatten() 
    
    #Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(signal).flatten()

    #Spectral centroid
    sc = librosa.feature.spectral_centroid(y=signal, sr=sample_rate).flatten()

    #Spectral Spread
    ss = librosa.feature.spectral_bandwidth(y=signal, sr=sample_rate).flatten()

    # Spectral Flux
    sf = librosa.onset.onset_strength(y=signal, sr=sample_rate).flatten()

    # Spectral Roll-Off
    srf = librosa.feature.spectral_rolloff(y=signal, sr=sample_rate).flatten()

    # Chroma Vector
    cv = librosa.feature.chroma_stft(y=signal, sr=sample_rate).flatten()

    #Chroma Deviation
    cv_mean = np.mean(cv, axis=1)  # Średnia dla każdego z 12 chroma
    cd = np.sqrt(np.mean((cv - cv_mean[:, np.newaxis])**2, axis=1))


    # Dodawanie do DataFrame
    temp = pd.DataFrame({'gwm': [gwm],"mfccs": [mfccs],"spectogram":[spectogram_DB],"zcr":[zcr],"sc":[sc],"ss":[ss],"sf":[sf],"srf":[srf],"cv":[cv],"cd":[cd], 'emotion': [emotion]})
    df = df._append(temp, ignore_index=True)
    
    #safety
    file_num += 1
    print(file_num)

In [ ]:
df.to_csv(r"../data/data_proccesed",index=False)

In [94]:
def extract_features(data, sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr))

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))
    
    return result

# def extract_features2(data, sample_rate):

#     result = np.array([])

#     zcr = librosa.feature.zero_crossing_rate(y=data).flatten()
#     result = np.hstack((result, zcr))

#     mfcc = librosa.feature.mfcc(y=data, sr=sample_rate).flatten()
#     result = np.hstack((result, mfcc))

#     mel = librosa.feature.melspectrogram(y=data, sr=sample_rate).flatten()
#     result = np.hstack((result, mel))

#     gwm = librosa.feature.rms(y=data).flatten() 
#     result = np.hstack((result, gwm))

#     return result

In [95]:
def get_features():
    data, sample_rate = librosa.load(os.path.join(data_folder, filename), duration=2.5, offset=0.6)
    
    res = extract_features(data, sample_rate)
    # res2 = extract_features2(data, sample_rate)
    # res = np.hstack((res, res2))
    result = np.array(res)
    

    return result

In [96]:
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw_file')
X, Y = [], []
for filename in os.listdir(data_folder):
    feature = get_features()
    file_parts = filename.split('-')
    emotion = int(file_parts[2])
    X.append(feature)
    Y.append(emotion)

In [97]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv(r"../data/data_proccesed.csv",index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,...,16353,16354,16355,16356,16357,16358,16359,16360,16361,labels
0,0.321275,0.729664,0.750032,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,...,0.000016,0.000012,0.000006,0.000008,0.000008,0.000008,0.000007,0.000006,0.000004,1
1,0.256113,0.621621,0.580278,0.598515,0.628296,0.598015,0.597272,0.650352,0.698635,0.681863,...,0.000031,0.000032,0.000028,0.000026,0.000031,0.000035,0.000035,0.000032,0.000024,1
2,0.207361,0.665823,0.670921,0.684690,0.707034,0.734071,0.654613,0.594455,0.626005,0.704741,...,0.000061,0.000052,0.000053,0.000052,0.000043,0.000044,0.000042,0.000035,0.000030,1
3,0.268903,0.601669,0.529868,0.497117,0.550246,0.579884,0.636897,0.690209,0.654453,0.590455,...,0.000012,0.000013,0.000011,0.000011,0.000012,0.000011,0.000011,0.000011,0.000008,1
4,0.163827,0.690411,0.748766,0.742984,0.678114,0.662585,0.633123,0.590499,0.698888,0.720797,...,0.000059,0.000055,0.000051,0.000026,0.000024,0.000023,0.000022,0.000020,0.000015,1


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [103]:
from tensorflow.keras import layers
from tensorflow.keras import models

normalizer = layers.Normalization()

normalizer.adapt(X)
model = models.Sequential([
    layers.Input(shape=(x_train.shape[1], 1)),
    normalizer,
    # layers.Reshape((18, 9)),
    layers.Conv1D(512, 1, activation='relu'),
    layers.Conv1D(256, 1, activation='relu'),
    layers.MaxPooling1D(),
    layers.Conv1D(128, 1, activation='relu'),
    layers.MaxPooling1D(),
    layers.Conv1D(64, 1, activation='relu'),
    layers.MaxPooling1D(),
    layers.Conv1D(32, 1, activation='relu'),
    layers.MaxPooling1D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax'),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics=['accuracy'],
)

In [107]:
model.fit(x_train, y_train, epochs=40, batch_size=2)

Epoch 1/40
153/576 [======>.......................] - ETA: 39s - loss: nan - accuracy: 0.0588

KeyboardInterrupt: 

In [93]:
model.evaluate(x_test,y_test)

9/9 [==============================] - 0s 2ms/step - loss: 2.0678 - accuracy: 0.3958


[2.06778883934021, 0.3958333432674408]